In [1]:
import os
# change dir to root
os.chdir('..')
os.getcwd()

'/Users/peter/repos/esc/emrecdem/emrecdem-framework'

In [2]:
import os
import subprocess
import codecs
import pandas as pd
import emrecdem as erd

import importlib
importlib.reload(erd)

TOPICS_DIR = "/Users/peter/repos/esc/data/Deniece/Textgrids_Timestamps_Participants_Face"
FEATURES_DIR = "/Users/peter/repos/esc/data/Deniece/openface-features"
AGG_FEATURES_DIR = "/Users/peter/repos/esc/data/Deniece/aggregated-features"
VIDEO_DIR = '/Volumes/vera_emrec/MEMOA_database'
#VIDEO_DIR = '/media/peter/Data/esc/emrecdem/MEMOA_database'

def process_video(video_dir, video_title):
    #cmd = f"/Users/peter/repos/esc/emrecdem/emrecdem-framework/notebooks/process_video.sh {video_dir} {video_title}"
    cmd = f"./process_video.sh {video_dir} {video_title}"
    print(cmd)
    #!{cmd}
    
def parseTime(s):
    split = s.split(':')
    minutes = int(split[0])
    seconds = float(split[1])
    total_seconds = 60 * minutes + seconds
    return total_seconds
    
def process_file(topics_file):
    (participant, session, experiment, memory, _) = topics_file.split("_")
    
    # Construct video path
    video_title = f'{participant}_{session}_{experiment}_{memory}_Cfront'
    video_dir = os.path.join(VIDEO_DIR, participant, session, 'Video', 'Single-angle')
    
    # Process video with openface
    process_video(video_dir, video_title)
    
    # Load openface features
    openface_file = os.path.join(FEATURES_DIR, video_title + '_features.csv')
    openface_features = pd.read_csv(openface_file, skipinitialspace=True)

    # Process topics from file
    aggregated_features = pd.DataFrame()
    topics_path = os.path.join(TOPICS_DIR, topics_file)
    with codecs.open(topics_path, 'r', encoding='cp1252') as file:
        for line in file:
            if not line.lower().startswith('timestamp'):
                continue
            
            # Extract start and end time, topic index and text
            line = line.strip()
            split = line.split(' ')
            start_time = parseTime(split[-2])
            end_time = parseTime(split[-1])
            duration = end_time - start_time
            topic_index = int(split[1].replace('.', ''))
            topic_label = ' '.join(split[2:-2])
            
            # Extract frames for this topic/fragment
            fragment_frames = erd.extract_fragment(openface_features, start_time, end_time)
            
            # Aggregate the values
            aggregated_row = erd.aggregate_frames(fragment_frames)
            
            # Combine results with information for this topic
            aggregated_row['start_time'] = start_time
            aggregated_row['end_time'] = end_time
            aggregated_row['duration'] = duration
            aggregated_row['topic_index'] = topic_index
            aggregated_row['topic_label'] = topic_label
            
            aggregated_features = pd.concat([aggregated_features, aggregated_row], ignore_index=True)
            
    output_file = "_".join([participant, session, experiment, memory]) + "_aggregated.csv"
    output_path = os.path.join(AGG_FEATURES_DIR, output_file)
    print(output_path)
    aggregated_features.to_csv(output_path, float_format='%g', index=False)
    return aggregated_features
    

# print(os.getcwd())
process_file('P1_S2_LSB_HM1_topics.txt')
#process_file('P2_S2_LSB_HM1_topics.txt')
#process_file('P11_S2_LSB_SM1_topics.txt')


./process_video.sh /Volumes/vera_emrec/MEMOA_database/P1/S2/Video/Single-angle P1_S2_LSB_HM1_Cfront
/Users/peter/repos/esc/data/Deniece/aggregated-features/P1_S2_LSB_HM1_aggregated.csv


,AU01_int_dur,AU01_int_dur_pmin,AU01_int_freq,AU01_int_freq_pmin,AU04_int_dur,AU04_int_dur_pmin,AU04_int_freq,AU04_int_freq_pmin,AU09_int_dur,AU09_int_dur_pmin,...,AU12_pres_freq_pmin,AU14_pres_dur,AU14_pres_dur_pmin,AU14_pres_freq,AU14_pres_freq_pmin,start_time,end_time,duration,topic_index,topic_label
0,0,0.0,0,0.0,0.00,0.000000,0,0.000000,0,0.0,...,1.355626,43.68,59.213737,1,1.355626,16.200,60.465,44.265,1,Beschrijving van het krijgen van eerste auto
1,0,0.0,0,0.0,0.00,0.000000,0,0.000000,0,0.0,...,0.000000,20.02,55.405904,3,8.302583,61.611,83.317,21.706,2,Stoer zijn
2,0,0.0,0,0.0,1.18,7.165992,1,6.072874,0,0.0,...,0.000000,9.88,60.000000,1,6.072874,83.317,93.220,9.903,3,Kosten auto
3,0,0.0,0,0.0,0.00,0.000000,0,0.000000,0,0.0,...,7.604563,15.66,59.543726,2,7.604563,94.327,110.123,15.796,4,Dankbaarheid voor de auto
4,0,0.0,0,0.0,0.00,0.000000,0,0.000000,0,0.0,...,5.763689,19.06,54.927954,3,8.645533,126.190,147.040,20.850,5,jaloezie van anderen over auto
5,0,0.0,0,0.0,0.00,0.000000,0,0.000000,0,0.0,...,2.384738,24.40,58.187599,4,9.538951,153.830,179.010,25.180,6,voor eerste keer de auto zien
6,0,0.0,0,0.0,0.00,0.000000,0,0.000000,0,0.0,...,5.774783,36.84,53.185756,8,11.549567,182.340,223.910,41.570,7,Eerste keer erin rijden
7,0,0.0,0,0.0,0.00,0.000000,0,0.000000,0,0.0,...,14.376997,11.72,56.166134,1,4.792332,226.120,238.660,12.540,8,Dankbaarheid naar ouders vergeten


In [ ]:
files = os.listdir(TOPICS_DIR)
files.sort()
for file in files:
    if file.endswith('.txt'):
        print("Processing", file)
        aggregated_features = process_file(file)